# Titanic: Machine Learning from Disaster

Author: Jingwen ZHENG<br>
Update: 2019-05-05

## Content
- Project understanding
- Objectif
- Practice skills
- Python packages to be applied
- Import data
- Data description
- Data cleaning
- Data analysis
- Build preprocessing pipeline
- Train data
- Reference

## Project understanding

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

## Objectif

In this challenge, we need to analyse what sorts of people were likely to survive.<br>
In particular, we also need to apply the tools of machine learning to predict which passengers survived the tragedy.

## Practice Skills

- Python 3
- Binary classification

## Python packages to be applied

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import reciprocal, uniform

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier


## Import data

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
print('Dimension train_df:', train_df.shape)
print('Dimension test_df:', test_df.shape)

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

## Data description

In [ ]:
train_df.describe(include='all').T

In [ ]:
train_df.info()

## Data cleaning

=> There are missing data in "Age", "Cabin" and "Embarked", especially "Cabin" whose 77% data are missing. So we will ignore it during the analysis.<br>
What should we do on missing data of "Age"? We might replace null by median age.<br>
Same for "Embarked"<br>
=> On average, the probability of one passenger being survived is 38%.

In [ ]:
train_df['Embarked'].value_counts()

In [ ]:
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
train_df['Embarked'].fillna('S', inplace=True)

In [ ]:
train_df.hist(bins=40, figsize=(18, 15))
plt.show()

## Data analysis

### Correlation matrix between numerical values and "Survived"

In [ ]:
sns.set(rc={'figure.figsize':(10, 8)})

sns.heatmap(train_df[['Survived', 'SibSp', 'Parch', 'Age', 'Pclass', 'Fare']].corr(),
            annot=True,
            fmt='.2f',
            cmap='coolwarm')
plt.show()

According to this correlation heatmap, we find that "Survived" possibility is slightly negative correlated with "Age", which means that the older the passenger is, the more possible he will be survived; it's significant at 5%. "Pclass" and "Fare are negatively correlated which is logic: the better class is, the more expensive fare is. "Pclass" is also negative correlated with "Survived" possibility, maybe the top class's location is nearer to exit?

### SibSp vs. Survived

In [ ]:
sibsp_survived_plt = sns.FacetGrid(train_df, col='Survived', height=4, aspect=1)
sibsp_survived_plt.map(sns.distplot, 'SibSp')

plt.show()

Most of survived people didn't have siblings / spouses aboard the Titanic, other survived majority is people who had one sibling / spouse aboard the Titanic. Very few people who had more than one sibling / spouse aboard the Titanic were survived, since it's difficult to survive all passengers.

In [ ]:
sibsp_sex_survived_plt = sns.factorplot(x='SibSp',
                                        col='Survived',
                                        data=train_df,
                                        hue='Sex',
                                        kind='count',
                                        palette='muted',
                                        size=4,
                                        aspect=1)
sibsp_sex_survived_plt.despine(left=True)
sibsp_sex_survived_plt.set_ylabels('Count')

plt.show()

Among all survivals, the majority was female.

### Parch vs. Survived

In [ ]:
parch_survived_plt = sns.FacetGrid(train_df, col='Survived', height=4, aspect=1)
parch_survived_plt.map(sns.distplot, 'Parch')

plt.show()

Similarly as relation between SibSp vs. Survived, most of survived people didn't have parents / children aboard the Titanic, other survived majority is people who had one parent / child aboard the Titanic. Very few people who had more than one parent / child aboard the Titanic were survived, since it's difficult to survive all passengers.

In [ ]:
parch_sex_survived_plt = sns.factorplot(x='Parch',
                                        col='Survived',
                                        data=train_df,
                                        hue='Sex',
                                        kind='count',
                                        palette='muted',
                                        size=4,
                                        aspect=1)
parch_sex_survived_plt.despine(left=True)
parch_sex_survived_plt.set_ylabels('Count')

plt.show()

Among all survivals, the majority was female.

### Age vs. Survived

In [ ]:
age_survived_plt = sns.FacetGrid(train_df, col='Survived', height=4, aspect=1)
age_survived_plt.map(sns.distplot, 'Age')

plt.show()

"Age" distribution is nearly a Normal distribution. We find that the most survived are the people around 25 year-old, which might because there were lots of younger on the board. However, people older than 50 year-old is less survived, which is not extractly same as analysis above. The graph below will explain why.

### Age vs. Sex vs. Survived

In [ ]:
age_sex_survived_plt = sns.FacetGrid(train_df , hue='Survived' , aspect=4 , row = 'Sex' )
age_sex_survived_plt.map( sns.kdeplot, 'Age' , shade= True )
age_sex_survived_plt.set( xlim=( 0 , train_df['Age'].max() ) )
age_sex_survived_plt.add_legend()
plt.show()

In [ ]:
train_df[(train_df['Survived']==0) & (train_df['Age']>50)].Sex.value_counts()

In [ ]:
train_df[(train_df['Survived']==1) & (train_df['Age']>50)].Sex.value_counts()

According to this plot, we find that both female and male passengers are nearly half survived, since the shadow surface of "Survived" and "Non-survived" are nearly the same. But the survived possibility is different on ages for female and male. For female passengers, the ones that older than 27 year-old have a little bit more chance to be survived than others. For male passengers, the one who is younger than 15 years old or older than 34 years old are more likely to be survived than men between 16 and 32 years old, which because they help other passengers to be alive.<br>
Moreover, let's pay attention to the passengers that were older than 50 year-old. According to the graph "Age vs. Survived" we find that people older than 50 year-old is less survived, which is not extractly same as correlation analysis. But from this graph we can easily understand why: most of older _passengers_ were male, most of the older _survival_ were female, and they were nearly all survived. So in fact, older female passengers had much more chance to be survived than older male passengers.

### Pclass vs. Survived

In [ ]:
pclass_survived_plt = sns.FacetGrid(train_df, col='Survived', height=4, aspect=1)
pclass_survived_plt.map(sns.distplot, 'Pclass')

plt.show()

For the class of survived passengers, it's almost same for each class, the possibility for class 1 is a bit more higher than others. However, non-survived probability of class 3 is 3 times of class 1 and class 2, respectively. This is coherent with analysis above.

### Embarked vs. Survived

In [ ]:
embarked_survived_plt = sns.factorplot(data=train_df,
                                       x='Embarked',
                                       y='Survived',
                                       size=6,
                                       kind='bar',
                                       palette='muted')
embarked_survived_plt.despine(left=True)
embarked_survived_plt.set_ylabels('Survival probability')

plt.show()

Passengers who came from Cherbourg (`Embarked='C'`) had more chance to be survived, so we'll study why they were most survived in the following.

In [ ]:
pclass_embarked_survived_plt = sns.factorplot(x='Pclass',
                                              col='Survived',
                                              data=train_df,
                                              hue='Embarked',
                                              kind='count',
                                              palette='muted',
                                              size=4,
                                              aspect=1)
pclass_embarked_survived_plt.despine(left=True)
pclass_embarked_survived_plt.set_ylabels('Count')

plt.show()

According to this graph, we find that the majority of passengers came from Southampton (`Embarked='S'`) and lots of them chose class 3 which decrease the survived chance. However, the marjority of passengers who came from Cherbourg (`Embarked='C'`) were in class 1, such their chance to be survived was more than others, which is coherent with result above.

## Build preprocessing pipeline
Inspired by https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb

In [ ]:
# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [ ]:
num_attribs = ['Age', 'SibSp', 'Parch', 'Fare']
cat_attribs = ['Pclass', 'Sex', 'Embarked']

### Build pipeline for numeric variables

In [ ]:
num_pipeline = Pipeline([
    ('select_numeric', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy='median'))
])

In [ ]:
num_pipeline.fit_transform(train_df)

### Build pipeline for categorical variables

In [ ]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [ ]:
cat_pipeline = Pipeline([
    ('select_numeric', DataFrameSelector(cat_attribs)),
    ('imputer', MostFrequentImputer()),
    ('cat_encoder', OneHotEncoder(sparse=False)) # return an array
])

In [ ]:
cat_pipeline.fit_transform(train_df)

### Join 2 pipelines into a single pipeline

In [ ]:
full_pipeline = FeatureUnion(transformer_list=[('num_pipeline', num_pipeline),
                                               ('cat_pipeline', cat_pipeline)]
                            )

In [ ]:
X_train = full_pipeline.fit_transform(train_df)
X_train

In [ ]:
y_train = train_df['Survived']

In [ ]:
X_test = full_pipeline.fit_transform(test_df)
X_test

## Train a classifier

### SVM (Support-vector machine)

In [ ]:
svr_params = {'kernel': ['rbf'],
              'gamma': [0.001, 0.01, 0.1, 1],
              'C': [1, 10, 50, 100, 200, 300, 1000]}
svr_gridsearch = GridSearchCV(SVC(random_state=42),
                              svr_params,
                              cv=StratifiedKFold(n_splits=10),
                              scoring='accuracy',
                              n_jobs=-1)
svr_gridsearch.fit(X_train, y_train)

In [ ]:
y_pred_svr = svr_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_svr)

### Decision Trees

In [ ]:
decisionTree_params = {'max_features': [1, 3, 10],
                       'min_samples_split': [2, 3, 10],
                       'min_samples_leaf': [1, 3, 10]}
decisionTree_gridsearch = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                       decisionTree_params,
                                       cv=StratifiedKFold(n_splits=10),
                                       scoring='accuracy',
                                       n_jobs=-1)
decisionTree_gridsearch.fit(X_train, y_train)

In [ ]:
y_pred_decisionTree = decisionTree_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_decisionTree)

### Random Forest

In [ ]:
rdmFrst_params = {'max_depth': [None],
                  'max_features': [1, 3, 10],
                  'min_samples_split': [2, 3, 10],
                  'min_samples_leaf': [1, 3, 10],
                  'bootstrap': [False],
                  'n_estimators':[100,300],
                  'criterion': ['gini']}
rdmFrst_gridsearch = GridSearchCV(RandomForestClassifier(random_state=42),
                                  rdmFrst_params,
                                  cv=StratifiedKFold(n_splits=10),
                                  scoring='accuracy',
                                  n_jobs=-1)
rdmFrst_gridsearch.fit(X_train, y_train)

In [ ]:
y_pred_rdmFrst = rdmFrst_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_rdmFrst)

### Logistic Regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
y_pred_logReg = log_reg.predict(X_train)
accuracy_score(y_train, y_pred_logReg)

### AdaBoost

In [ ]:
# Inspired by https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
ada_params = {'base_estimator__criterion': ['gini', 'entropy'],
              'base_estimator__splitter': ['best', 'random'],
              'algorithm': ['SAMME', 'SAMME.R'],
              'n_estimators': [1, 2],
              'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 1.5]}

ada_gridsearch = GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier(random_state=42)),
                              param_grid = ada_params,
                              cv=StratifiedKFold(n_splits=10),
                              scoring='accuracy',
                              n_jobs= -1)

ada_gridsearch.fit(X_train, y_train)

In [ ]:
ada_gridsearch.best_score_

In [ ]:
y_pred_ada = ada_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_ada)

### Extra Trees

In [ ]:
# Inspired by https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
extraTree_params = {'max_depth': [None],
                    'max_features': [1, 3, 10],
                    'min_samples_split': [2, 3, 10],
                    'min_samples_leaf': [1, 3, 10],
                    'bootstrap': [False],
                    'n_estimators': [100, 300],
                    'criterion': ['gini']}
extraTree_gridsearch = GridSearchCV(ExtraTreesClassifier(random_state=42),
                                    extraTree_params,
                                    cv=StratifiedKFold(n_splits=10),
                                    scoring='accuracy',
                                    n_jobs=-1)
extraTree_gridsearch.fit(X_train, y_train)

In [ ]:
extraTree_gridsearch.best_score_

In [ ]:
y_pred_extraTree = extraTree_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_extraTree)

### Gradient Boosting

In [ ]:
# Inspired by https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
gbrt_params = {'n_estimators': [100, 200, 300],
               'loss': ['deviance'],
               'learning_rate': [0.01, 0.05, 0.1],
               'max_depth': [4, 8],
               'min_samples_leaf': [100, 150],
               'max_features': [0.1, 0.3]}
gbrt_gridsearch = GridSearchCV(GradientBoostingClassifier(random_state=42),
                               gbrt_params,
                               cv=StratifiedKFold(n_splits=10),
                               scoring='accuracy',
                               n_jobs=-1)
gbrt_gridsearch.fit(X_train, y_train)

In [ ]:
gbrt_gridsearch.best_score_

In [ ]:
y_pred_gbrt = gbrt_gridsearch.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred_gbrt)

### Voting classifier

In [ ]:
voting_hard_clf = VotingClassifier(
    estimators=[('svm', svr_gridsearch.best_estimator_),
                ('dt', decisionTree_gridsearch.best_estimator_),
                ('rf', rdmFrst_gridsearch.best_estimator_),
                ('lr', log_reg),
                ('ada', ada_gridsearch.best_estimator_),
                ('extraTree', extraTree_gridsearch.best_estimator_),
                ('gbrt', gbrt_gridsearch.best_estimator_)],
    voting='hard'
)

voting_hard_clf.fit(X_train, y_train)

In [ ]:
y_pred_voting_hard = voting_hard_clf.predict(X_train)
accuracy_score(y_train, y_pred_voting_hard)

In [ ]:
voting_hard_clf_90 = VotingClassifier(
    estimators=[('svm', svr_gridsearch.best_estimator_),
                ('rf', rdmFrst_gridsearch.best_estimator_),
                ('ada', ada_gridsearch.best_estimator_),
                ('extraTree', extraTree_gridsearch.best_estimator_),
                ('gbrt', gbrt_gridsearch.best_estimator_)],
    voting='hard'
)

voting_hard_clf_90.fit(X_train, y_train)

In [ ]:
y_pred_voting_hard_90 = voting_hard_clf_90.predict(X_train)
accuracy_score(y_train, y_pred_voting_hard_90)

In [ ]:
svr_gridsearch.best_estimator_.probability = True
voting_soft_clf = VotingClassifier(
    estimators=[('svm', svr_gridsearch.best_estimator_),
                ('dt', decisionTree_gridsearch.best_estimator_),
                ('rf', rdmFrst_gridsearch.best_estimator_),
                ('lr', log_reg),
                ('ada', ada_gridsearch.best_estimator_),
                ('extraTree', extraTree_gridsearch.best_estimator_),
                ('gbrt', gbrt_gridsearch.best_estimator_)],
    voting='soft'
)

voting_soft_clf.fit(X_train, y_train)

In [ ]:
y_pred_voting_soft = voting_soft_clf.predict(X_train)
accuracy_score(y_train, y_pred_voting_soft)

### Compare different models' accuracy score

In [ ]:
accuracy_score_df = pd.DataFrame({'Models': ['SVM',
                                             'Decision Trees',
                                             'Random Forest',
                                             'Logistic Regression',
                                             'AdaBoost',
                                             'Extra Trees',
                                             'Gradient Boosting',
                                             'Hard voting',
                                             'Hard voting gt 90pct',
                                             'Soft voting'],
                                  'Accuracy score': [accuracy_score(y_train, y_pred_svr),
                                                     accuracy_score(y_train, y_pred_decisionTree),
                                                     accuracy_score(y_train, y_pred_rdmFrst),
                                                     accuracy_score(y_train, y_pred_logReg),
                                                     accuracy_score(y_train, y_pred_ada),
                                                     accuracy_score(y_train, y_pred_extraTree),
                                                     accuracy_score(y_train, y_pred_gbrt),
                                                     accuracy_score(y_train, y_pred_voting_hard),
                                                     accuracy_score(y_train, y_pred_voting_hard_90),
                                                     accuracy_score(y_train, y_pred_voting_soft)]},
                                 columns=['Models', 'Accuracy score'])
accuracy_score_df.sort_values('Accuracy score', inplace=True, ascending=False)
accuracy_score_df.reset_index(inplace=True, drop=True)

In [ ]:
accuracy_score_df

In [ ]:
plt.figure(figsize=(10, 5))
plt.barh(np.arange(len(accuracy_score_df['Models'])),
         accuracy_score_df['Accuracy score'],
         align='center',
         height=0.5)

plt.yticks(np.arange(len(accuracy_score_df['Models'])), accuracy_score_df['Models'])
plt.tick_params(labelsize=12)
plt.xlabel('Accuracy score', fontdict={'fontsize': 13})
plt.ylabel('Models', fontdict={'fontsize': 13})

plt.show()

### Compare different models' best_score_

In [ ]:
best_score_df = pd.DataFrame({'Models': ['SVM',
                                         'Decision Trees',
                                         'Random Forest',
                                         'AdaBoost',
                                         'Extra Trees',
                                         'Gradient Boosting'],
                              'best_score_': [svr_gridsearch.best_score_,
                                              decisionTree_gridsearch.best_score_,
                                              rdmFrst_gridsearch.best_score_,
                                              ada_gridsearch.best_score_,
                                              extraTree_gridsearch.best_score_,
                                              gbrt_gridsearch.best_score_]},
                             columns=['Models', 'best_score_'])
best_score_df.sort_values('best_score_', inplace=True, ascending=False)
best_score_df.reset_index(inplace=True, drop=True)

In [ ]:
best_score_df

In [ ]:
plt.figure(figsize=(10, 5))
plt.barh(np.arange(len(best_score_df['Models'])), best_score_df['best_score_'], align='center', height=0.5)

plt.yticks(np.arange(len(best_score_df['Models'])), best_score_df['Models'])
plt.tick_params(labelsize=12)
plt.xlabel('best_score_', fontdict={'fontsize': 13})
plt.ylabel('Models', fontdict={'fontsize': 13})

plt.show()

## Prediction with test data

Since the root MSE of Random Forest model is the least among four models that we chose, I'll apply Random Forest to predict survivals in test dataset.

In [ ]:
y_test = voting_soft_clf.predict(X_test)

In [ ]:
passengerID_test = test_df['PassengerId']
output_df = pd.DataFrame({'PassengerId':passengerID_test,
                          'Survived': y_test},
                         columns=['PassengerId', 'Survived'])

In [ ]:
output_df.head()

In [ ]:
len(output_df)

In [ ]:
output_df.to_csv('titanic_predict_survivals.csv', index=False)

## Reference

- Kaggle Competition: [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/overview)
- [Introduction to Ensembling/Stacking in Python](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python)
- [Titanic Top 4% with ensemble modeling](https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling)
- [An Interactive Data Science Tutorial](https://www.kaggle.com/helgejo/an-interactive-data-science-tutorial)
- [handson-ml/03_classification.ipynb](https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb)